# Earthquake location as an inverse problem

[![Open In Colab](https://img.shields.io/badge/open%20in-Colab-b5e2fa?logo=googlecolab&style=flat-square&color=ffd670)](https://colab.research.google.com/github/tsonpham/ObsSeisHUS2025/blob/master/Day3/D3_Prac.ipynb)

Prepared by Thanh-Son Pham (thanhson.pham@anu.edu.au), April 2025.

---
## What we do in this notebook

Here we demonstrate a real inverse problem example of determining the earthquake hypocentre using  seismic data,
- Cast the hypocentre determination for as an inverse problem
- Solve the inverse problem in the optimal framework, by minimizing a misfit function
- Solve the inverse problem in the Bayesian framework with the advanced ensemble samplers

In [ ]:
# Environemtal setup (uncomment if running in colab)

# !pip install emcee obspy arviz numpy==1.26.4 basemap corner

In [ ]:
#@title Setting notebook resolution
#@markdown Run this cell for better figure resolution

%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams

rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 10

---
## Problem setup

We download some waveform data from the IRIS server in preparation for the problem.

In [ ]:
#@title Run to download example seismic data
#@markdown Let start with download some waveform data of the M5.2 Kon Tum 28/07/2024 earthquake to demonstrate the example. This cell downloads seismic data from the IRIS database using the `mass_downloader`. For more instruction, see Day2 self-practice [excercise](https://colab.research.google.com/github/tsonpham/ObsSeis-VNU/blob/master/Day2/D2_Prac.ipynb) to learn more.

from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader

# Initialize the client for ISC
isc = Client("ISC")
# Request event information
event = isc.get_events(eventid="641665444")[0]
origin_time = event.preferred_origin().time
origin_lat = event.preferred_origin().latitude
origin_lon = event.preferred_origin().longitude

# Circular domain around the epicenter
domain = CircularDomain(origin_lat, origin_lon, minradius=0.0, maxradius=15.0)
# Restriction on the waveform data
restrictions = Restrictions(
    starttime=origin_time - 1 * 60,
    endtime=origin_time + 10 * 60,
    reject_channels_with_gaps=True,
    minimum_length=0.95,
    minimum_interstation_distance_in_m=150E3,
    channel_priorities=["BH[ZNE12XY]", "HH[ZNE]"],
    location_priorities=["", "00", "10"])

# Initialize the mass downloader with specific providers
mdl = MassDownloader(providers=['IRIS'])
mdl.download(domain, restrictions, mseed_storage="waveforms", stationxml_storage="stations")

In [ ]:
#@title Run to read waveform and metadata
#@markdown This cell reads the downloaded data and creates an Inventory `inv` and Stream `dstream` objects for meta and waveform data.

from obspy import read_inventory, read, Inventory, Stream
from pathlib import Path
## Read all the stationxml files and merge them into one Inventory object
inv = Inventory()
for file in Path("stations").glob("*.xml"): inv += read_inventory(str(file))
## Read all the waveform files and merge them into one Stream object
dstream = Stream()
for file in Path("waveforms").glob("*.mseed"): dstream += read(str(file))

In [ ]:
#@title Run to plot the station map
#@markdown Let's plot the event location (available in the ISC bulletin, event id [641665444](https://isc.ac.uk/cgi-bin/web-db-run?event_id=641665444&out_format=ISF2&request=COMPREHENSIVE) and available seismic stations on the map. The `plot_map` funtion is defined to plot the event configuration at several occasions.

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from obspy.geodetics import locations2degrees

def plot_map(event, inv, obs_data=None, contour=False):
    # create a new figure and axis
    fig, ax = plt.subplots()
    # initialize the basemap, specifing the projection, the gegraphic buondary, and resolution
    # the initialzed map instance is attached to the axis
    m = Basemap(projection='merc',ax=ax, llcrnrlat=0, urcrnrlat=26, llcrnrlon=94, urcrnrlon=122, resolution='l')
    # draw coastlines
    m.drawcoastlines(linewidth=0.75)
    # draw country boundaries
    m.drawcountries(linewidth=0.75)
    # draw parallels and meridians
    m.drawparallels(range(0, 25, 5), labels=[1,0,0,0], linewidth=0.3, color='gray', dashes=(5, 3))
    m.drawmeridians(range(90, 125, 5), labels=[0,0,0,1], linewidth=0.3, color='gray', dashes=(5, 3))
    # plot the epicenter
    m.plot(event.preferred_origin().longitude, event.preferred_origin().latitude,
           'rx', markersize=12, label='ISC location', latlon=True)
    # plot the station locations
    for network in inv:
        for station in network:
            lon = station.longitude
            lat = station.latitude
            # mark the station location
            m.plot(lon, lat, '^', c='gray', markersize=4, latlon=True)
            # put the station label
            x, y = m(lon, lat)
            ax.text(x, y+2.5e4, station.code, fontsize=8, color='gray', ha='center',
                    bbox=dict(fc='white', ec='none', boxstyle='round,pad=0.0'))
    # plot station in the observed data
    if obs_data:
        for k, v in obs_data.items():
            # mark the station location
            m.plot(v['lon'], v['lat'], '^', c='blue', markersize=4, latlon=True)
            # put the station label
            x, y = m(v['lon'], v['lat'])
            ax.text(x, y+2.5e4, k, fontsize=8, color='blue', ha='center',
                    bbox=dict(fc='white', ec='none', boxstyle='round,pad=0.0'))

    if contour: # plot the distance contours to the epicenter
        # plot distance contours to the epicenter
        x = np.linspace(m.xmin, m.xmax, 300)
        y = np.linspace(m.ymin, m.ymax, 300)
        mlon, mlat = m(*np.meshgrid(x, y), inverse=True)
        dist = locations2degrees(origin_lat, origin_lon, mlat, mlon)
        c = m.contour(mlon, mlat, dist, levels=range(0, 19, 3), latlon=True, colors='k', linewidths=0.5)
        plt.clabel(c, inline=1, fontsize=8, fmt='%d°', colors='k')
    # return the map instance
    return m

# plot the station map
m = plot_map(event, inv, contour=True)
m.ax.legend(loc='upper right')
plt.show()

---
## Seismic observations: Differences of S- and P-wave arrivals

We reuse the measurements of P and S wave arrivals from Day 2 self-practice [excercise](https://isc.ac.uk/cgi-bin/web-db-run?event_id=641665444&out_format=ISF2&request=COMPREHENSIVE) encapsulated in a dictionary named `obs_data1`. Each dictionary entry corresponding to a station containting the station's coordinates, and P-, S-wave arrival times and their difference.

In [ ]:
from obspy import UTCDateTime

# Picked P and S arrival times for stations QIZ, VIVO, and PBKT as done in the Day 2 practical notebook
obs_data1 = {
    'QIZ':  {'parv': UTCDateTime(2024,7,28,4,36,20), 'sarv': UTCDateTime(2024,7,28,4,37,9)},
    'VIVO': {'parv': UTCDateTime(2024,7,28,4,36,37), 'sarv': UTCDateTime(2024,7,28,4,37,35)},
    'PBKT': {'parv': UTCDateTime(2024,7,28,4,37,0), 'sarv': UTCDateTime(2024,7,28,4,38,20)}
    }

# update the station coordinates from the inventory
for key, value in obs_data1.items():
    tmp = inv.select(station=key)
    obs_data1[key].update({'lat': tmp[0][0].latitude, 'lon': tmp[0][0].longitude,
                           # S-P differential arrival time
                           'tdiff': value['sarv'] - value['parv']})

In [ ]:
#@title Click to visualze seismic observations
#@markdown This cell plots data and their picked P- and S-wave arrivals for quality assurance.

def plot_waveform(stacode, obs_data, filter_kw=dict(type='highpass', freq=0.1, corners=2, zerophase=False)):
    # plot the waveform
    st = dstream.select(station=stacode)
    # rotate the 3 orthogonal component seismograms to vertical (Z), north (N), and east (E) directions
    st.rotate('->ZNE', inventory=inv)
    # view the first 5 minutes of the seismograms from the origin time
    st.trim(origin_time, origin_time+300)
    # sequence to detrend, taper, and filter the seismoograms
    st.detrend('demean')
    st.taper(max_percentage=0.05)
    st.filter(**filter_kw)
    # plot the seismograms
    fig = plt.figure()
    st.plot(fig=fig)
    # mark the P and S arrival times
    for ax in fig.axes:
        # mark the P and S arrival times
        ax.plot(obs_data[stacode]['parv'].datetime, 0, 'r|', markersize=20, label='P-arrival')
        try:
            ax.plot(obs_data[stacode]['sarv'].datetime, 0, 'b|', markersize=20, label='S-arrival')
        except: # if the S-arrival is not provided
            pass
        # set the y-axis label
        ax.set(ylabel='Amplitude', yticks=[0])

    # show the figure legend
    ax.legend()
    plt.show()

## plot the seismograms
for key in obs_data1.keys():
    plot_waveform(key, obs_data1)

### Definite the forward problem

The forward problem inputs an event coordinate (lat and lon) to predict the S- to P-wave travel time difference to each station in the dataset, specifically,
- input: hypocentral coordinates $(\theta, \phi)$
- ouput: differential travel time of S- and P- wave arrivals

The forward problem is hard to be expressed analytically. In seismology, it is common to use numerical solvers to define a forward problem.

Here, we use the `taup` to predict travel times of seismic phases in spherial Earth models. The package was origninally written in the Java programming language, which is packaged in `obspy` to allow pythonic interface.

In [ ]:
from obspy.taup import TauPyModel
taup_model = TauPyModel(model="ak135")

## calculate the theoretical P and S arrival times for the stations
precomp_d = np.arange(0, 20.1, 0.5)
precomp_tp = np.zeros_like(precomp_d)
precomp_ts = np.zeros_like(precomp_d)
for i, d in enumerate(precomp_d):
    arrivals = taup_model.get_travel_times(0, d, ['P'])
    precomp_tp[i] = arrivals[0].time
    arrivals = taup_model.get_travel_times(0, d, ['S'])
    precomp_ts[i] = arrivals[0].time

For each hypocentral coordinate, the P and S travel time to each stations measured are computed using the taup `get_travel_times_geo` function. The prediction for each hypocentra will be S- to P- travel time differences corresponding to the observed stations.

In [ ]:
def forward_prob1(S):
    '''
    This forward problem returns the differential travel times of the S wave
    and P wave for a given epicenter location S observed at three stations
    QIZ, VIVO, PBKT.
    '''
    src_lat, src_lon = S
    output = []
    for rcv in obs_data1.values():
        # distance from receiver to source
        d = locations2degrees(src_lat, src_lon, rcv['lat'], rcv['lon'])
        # calculate the theoretical P and S arrival times
        t = np.interp(d, precomp_d, precomp_ts) - np.interp(d, precomp_d, precomp_tp)
        output.append(t)
    return np.array(output)

### Optimal inverse solution

First, we try to find the inverse solution by minizing the misfit function. Here we use the `Nelder-Mead` algoritm from a list of built-in [optimizers](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) from the `scipy` package.

In [ ]:
## pre-defined optimizer
from scipy.optimize import minimize

dt = np.array([v['tdiff'] for v in obs_data1.values()])
## define the objective function
def objective_function(S):
    pred = forward_prob1(S)
    return np.sum((dt - pred)**2)

## optimize the objective function using the Nelder-Mead method
S0 = (origin_lat, origin_lon)
res = minimize(objective_function, x0=S0, method='Nelder-Mead')

## print the result with defined format
print (f'Optimized epicenter location: Latitude = {res.x[0]:.4f}, Longitude = {res.x[1]:.4f}')
print (f'Success: {res.success}')
print (f'Message: {res.message}')
print (f'Number of function evaluations: {res.nfev}')
print (f'Number of iterations: {res.nit}')
print (f'Objective function value: {res.fun:.4f}')

The optimal solution looks reasonably good on a map. However, we know that the inverse solution is subjected to some uncertainty, how can we quantify it?

In [ ]:
m = plot_map(event, inv, obs_data1)
m.plot(res.x[1], res.x[0], '+g', markersize=12, label='Optimized', latlon=True)
m.ax.legend(loc='upper right')
plt.show()

### Bayesian sampling and ensemble solutions

Now, we seek to find the ensemble of possible solution in form of the posterior distribution in the Bayesian approach.
- The prior function defines a geographical box of earthquake lat, lon, where the proabiblity of event location is uniformly defined.
- The likelihood assumes Gaussian data noise for three time difference measurements.

In [ ]:
lower_bound = (8, 94)
upper_bound = (24, 118)

def log_prior1(S):
    '''
    This function computes the prior probability of the epicenter location S.
    '''
    if not (lower_bound[0] <= S[0] <= upper_bound[0] and lower_bound[1] <= S[1] <= upper_bound[1]):
        return -np.inf
    return 0 # uninformative prior within the map boundary

def log_likelihood1(S, sigma=5):
    '''
    This function computes the log likelihood of the observed data given the
    epicenter location S.
    '''
    pred = forward_prob1(S)
    sigma = np.ones_like(obs_data1) * sigma
    return -.5 * np.sum((dt - pred)**2 / sigma**2 + np.log(2 * np.pi * sigma**2)) # Gaussian likelihood

def log_prob1(X):
    '''
    This function computes the log likelihood of the observed data given an coordinate
    in the parameter space.
    '''
    return log_prior1(X) + log_likelihood1(X, 10)

## initialize the walkers
nsteps = 400
nwalkers = 32

## number of dimensions, which is 2 for lat and lon of the epicenter
ndim = 2
walker_start = np.random.uniform(0, 1, (nwalkers, ndim))
for i in range(ndim): # uniform start
    walker_start[:, i] = walker_start[:, i] * (upper_bound[i] - lower_bound[i]) + lower_bound[i]

## run the MCMC
import emcee
sampler1 = emcee.EnsembleSampler(nwalkers, ndim, log_prob_fn=log_prob1)
output1 = sampler1.run_mcmc(walker_start, nsteps, progress=True)

The trace evolutions of invididual parameters look as below.

In [ ]:
import arviz as az
idata1 = az.from_emcee(sampler1, var_names=['lat', 'lon'])
ax = az.plot_trace(idata1)
plt.tight_layout()
plt.show()

The posterior confirm that the ISC location and the optimal solution (found above) are likely solutions. However, they also reveal other possible solutions up North (at the Vietnam-China boder). It is possibly due to the configuration of three observing stations. Do you have any thought on how to improve the configuration?

In [ ]:
x, y = m(idata1.posterior.lon.values, idata1.posterior.lat.values)
m = plot_map(event, inv, obs_data1)
az.plot_pair({'': x.flatten(), ' ': y.flatten()}, kind='kde', ax=m.ax,
            kde_kwargs={'contour': False, 'pcolormesh_kwargs': {'cmap': 'BuGn'}})
m.ax.set_title('Posterior distribution of epicenter location')
m.ax.legend(loc='upper right')
plt.show()

### Hierachical Bayesian sampling

Similar to the in-class exercise, we here employ the hierarchical approach to let the data decides their own noise level by introducing the hyperparameter, $\sigma$, of noise level.

In [ ]:
lower_bound = (8, 94, 0)
upper_bound = (24, 118, 15)

def log_prior1b(X):
    '''
    This function computes the prior probability of the epicenter location S.
    '''
    for i in range(len(X)):
        if not (lower_bound[i] <= X[i] <= upper_bound[i]):
            return -np.inf
    return 0 # uninformative prior within the map boundary

def log_likelihood1b(X):
    '''
    This function computes the log likelihood of the observed data given the
    epicenter location S.
    '''
    pred = forward_prob1(X[:2])
    sigma = np.ones_like(dt) * X[2]
    return -.5 * np.sum((dt - pred)**2 / sigma**2 + np.log(2 * np.pi * sigma**2)) # Gaussian likelihood

def log_prob1b(X):
    '''
    This function computes the log likelihood of the observed data given an coordinate
    in the parameter space.
    '''
    return log_prior1b(X) + log_likelihood1b(X)

## initialize the walkers
nsteps = 1000
nwalkers = 32

## number of dimensions, which is 2 for lat and lon of the epicenter
ndim = 3
walker_start = np.random.uniform(0, 1, (nwalkers, ndim))
for i in range(ndim): # uniform start
    walker_start[:, i] = walker_start[:, i] * (upper_bound[i] - lower_bound[i]) + lower_bound[i]

## run the MCMC
sampler1b = emcee.EnsembleSampler(nwalkers, ndim, log_prob_fn=log_prob1b)
output1b = sampler1b.run_mcmc(walker_start, nsteps, progress=True)

In [ ]:
idata1b = az.from_emcee(sampler1b, var_names=['lat', 'lon', 'sigma'])
ax = az.plot_trace(idata1b)
plt.tight_layout()
plt.show()

The hypocentre location seem to converse well, but the hyperparameter, $\sigma$, does not. This is because we only have three data point. It is hard to robustly separate 'signal' from 'noise' for such a small population.

In [ ]:
x, y = m(idata1b.posterior.lon.values, idata1b.posterior.lat.values)
m = plot_map(event, inv, obs_data1)
az.plot_pair({'': x.flatten(), ' ': y.flatten()}, kind='kde', ax=m.ax,
            kde_kwargs={'contour': False, 'pcolormesh_kwargs': {'cmap': 'BuGn'}})
m.ax.set_title('Posterior distribution of epicenter location')
m.ax.legend(loc='upper right')
plt.show()

---
## Seismic observations: first P-wave arrivals

You might have noticed that S-wave arrivals of this event are often noisy and hard to pick. Here we propose an alternative way to employ observation from more stations providing better azimuthal coverage to the source.

P-wave arrivals are often easy to pick for more stations. However, one problem with P-wave arrivals only is that they are subjected to systematic errors due to the unknown event origin time. Our proposed solution is to use pairwise travel time differences. It minimizes the unceratinty assosiating to the unknown orign time.

In [ ]:
obs_data2 = {
    'TWGB': {'parv': UTCDateTime(2024,7,28,4,38,40)},
    'SBM': {'parv': UTCDateTime(2024,7,28,4,38,19)},
    'KKM': {'parv': UTCDateTime(2024,7,28,4,38,3)},
    'QIZ':  {'parv': UTCDateTime(2024,7,28,4,36,20)},
    'VIVO': {'parv': UTCDateTime(2024,7,28,4,36,37)},
    'PBKT': {'parv': UTCDateTime(2024,7,28,4,37,0)},
    }

# update the station coordinates from the inventory
for key, rcv in obs_data2.items():
    tmp = inv.select(station=key)
    obs_data2[key].update({'lat': tmp[0][0].latitude, 'lon': tmp[0][0].longitude})

## plot the seismograms
for key in obs_data2.keys():
    plot_waveform(key, obs_data2)#, filter_kw=dict(type='bandpass', freqmin=0.1, freqmax=1.0, corners=2, zerophase=False))


Because the observed data have changed, we modify the forward problem predict the pair-wise travel time differences between observing stations.

In [ ]:
lower_bound = (8, 100, 0)
upper_bound = (24, 115, 15)

def forward_prob2(S):
    '''
    This forward problem returns the differential travel times of the S wave
    and P wave for a given epicenter location S observed at three stations
    QIZ, VIVO, PBKT.
    '''
    src_lat, src_lon = S[:2]
    tp_pred = []
    for rcv in obs_data2.values():
        # distance from receiver to source
        d = locations2degrees(src_lat, src_lon, rcv['lat'], rcv['lon'])
        # calculate the theoretical P and S arrival times
        tp_pred.append(np.interp(d, precomp_d, precomp_tp))
    return  np.array(tp_pred)

def log_likelihood2(X):
    '''
    This function computes the log likelihood of the observed data given the
    epicenter location S.
    '''
    tp_pred = forward_prob2(X)
    tp = np.array([v['parv'] for v in obs_data2.values()])

    idx = np.triu_indices(len(tp_pred), 1)

    tp_pred = tp_pred[idx[0]] - tp_pred[idx[1]]
    # print ('pred', tp_pred.astype(int))
    tp = tp[idx[0]] - tp[idx[1]]
    # print ('obs', tp.astype(int))

    sigma = X[2] * np.ones_like(tp_pred)
    return -.5 * np.sum((tp - tp_pred)**2 / sigma**2 + np.log(2 * np.pi * sigma**2)) # Gaussian likelihood

def log_prob2(X):
    '''
    This function computes the log likelihood of the observed data given an coordinate
    in the parameter space.
    '''
    return log_prior1b(X) + log_likelihood2(X)

In [ ]:
## initialize the walkers
nsteps = 5000
nwalkers = 64

## number of dimensions, which is 2 for lat and lon of the epicenter
ndim = 3
walker_start = np.random.uniform(0, 1, (nwalkers, ndim))
for i in range(ndim): # uniform start
    walker_start[:, i] = walker_start[:, i] * (upper_bound[i] - lower_bound[i]) + lower_bound[i]

## run the MCMC
sampler2 = emcee.EnsembleSampler(nwalkers, ndim, log_prob_fn=log_prob2)
output2 = sampler2.run_mcmc(walker_start, nsteps, progress=True)

## plot the sampling traces
idata2 = az.from_emcee(sampler2, var_names=['lat', 'lon', 'sigma'])
ax = az.plot_trace(idata2)
plt.tight_layout()

## source location distribution on map
x, y = m(idata2.posterior.lon.values, idata2.posterior.lat.values)
m = plot_map(event, inv, obs_data2)
az.plot_pair({'': x.flatten(), ' ': y.flatten()}, kind='kde', ax=m.ax, kde_kwargs={
            'contour': False, 'pcolormesh_kwargs': {'cmap': 'BuGn'}})
m.ax.set_title('Posterior distribution of epicenter location')
m.ax.legend(loc='upper right')
m.plot(event.preferred_origin().longitude, event.preferred_origin().latitude,
       'rx', markersize=12, label='Catalog', latlon=True)
plt.show()

In [ ]:
######## autocorrelation time
tau2 = sampler2.get_autocorr_time()
print(f"autocorrelation time: {tau2}")

######## corner plot to show pair-marginalized posterior distributions
import corner
flat_samples = sampler2.get_chain(discard=int(tau2.max()), thin=int(tau2.max()*.5), flat=True)
fig = corner.corner(flat_samples, #labels=labels, #truths=list(true_model) + [sigma],
                        quantiles=[0.025, 0.5, 0.975], show_titles=True,
                        levels=(0.68, 0.95), alpha=0.1)
fig.set_size_inches(7, 7)
plt.show()


---
## Challenge

Currently we have picked P-wave arrivals for six stations recording the M5.2 Kon Tum 28/07/2024 earthquake. Would you try to pick the arrival time for a couple more stations and re-run the inversion?

---
## Remarks

- Earthquake location is a classical inverse problem in seismology. We have worked with arrival time and casted our forward problem on ray theory for travel time predictions. There's a class of algorithm employing full waveform for earthquake location.

- Dealing with real data requires flexibility and creation to over come limitation posed by the available data.